# Datamining Program for Procurement

The following piece of code acquires a data frame for all procurement contracts related to a single keyword. The first entry makes a function to get all the links for the contracts.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

driver =  webdriver.Chrome('/Users/vatsalbajaj/Downloads/chromedriver')
old_links = []
def get_links(num):
    try:
        website_name = 'https://eprocure.gov.in/cppp/resultoftenders/byJWFieSVic29mdHdhcmVieSVjYnklZHNlbGVjdGJ5JWUyMDE3YnklZlB1Ymxpc2hlZA==/page='
        page_num = str(num)
        driver.get(website_name + page_num)
        table = driver.find_element_by_id('edit-l-result-teners')
        tender = table.get_attribute('innerHTML')
        semi_links = [m.start() for m in re.finditer('a href="/cppp/aocfullview', tender)]
        for semilink in semi_links:
            old_links.append('https://eprocure.gov.in' + tender[semilink + 8: semilink+38])
    except:
        captcha_URL = driver.current_url
        driver.get(captcha_URL)
        captcha_problem = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/span").text
        answer = int(captcha_problem[0:2]) + int(captcha_problem[-4:-2])
        inputting_location = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/input")
        inputting_location.send_keys(answer)
        driver.find_element_by_xpath("//form/div/input").click()
        get_links(num)

The next entry makes a list of links and prints them out.

In [3]:
for x in range(1,36):
    get_links(x)
links = []
for elem in old_links:
    link = elem.replace('"', '')
    links.append(link)
for link in links:
    print(link)

https://eprocure.gov.in/cppp/aocfullview/id%3D992117
https://eprocure.gov.in/cppp/aocfullview/id%3D1005037
https://eprocure.gov.in/cppp/aocfullview/id%3D1086094
https://eprocure.gov.in/cppp/aocfullview/id%3D996046
https://eprocure.gov.in/cppp/aocfullview/id%3D1121907
https://eprocure.gov.in/cppp/aocfullview/id%3D1122607
https://eprocure.gov.in/cppp/aocfullview/id%3D1090427
https://eprocure.gov.in/cppp/aocfullview/id%3D996338
https://eprocure.gov.in/cppp/aocfullview/id%3D1116246
https://eprocure.gov.in/cppp/aocfullview/id%3D915868
https://eprocure.gov.in/cppp/aocfullview/id%3D1132119
https://eprocure.gov.in/cppp/aocfullview/id%3D1123151
https://eprocure.gov.in/cppp/aocfullview/id%3D967353
https://eprocure.gov.in/cppp/aocfullview/id%3D1033653
https://eprocure.gov.in/cppp/aocfullview/id%3D914572
https://eprocure.gov.in/cppp/aocfullview/id%3D914571
https://eprocure.gov.in/cppp/aocfullview/id%3D1003256
https://eprocure.gov.in/cppp/aocfullview/id%3D1016896
https://eprocure.gov.in/cppp/aocful

The next entry creates a utility function to manipulate date and time, and it also creates a contract class.

In [4]:
import datetime

def convert_date(date):
        dt_obj = (datetime.datetime.strptime(date, "%d-%b-%Y %I:%M %p"))
        unix_time = dt_obj.timestamp()
        return unix_time
    
class Contract():
    
    def __init__(self, organisation_name, tender_description, tender_type, number_of_bids, name_of_selected_bidder, contract_value, published_date, contract_date, completion_date):
        self.organisation_name = organisation_name.text
        self.tender = tender_description.text
        self.type = tender_type.text
        self.number = number_of_bids.text
        self.winner = name_of_selected_bidder.text
        self.value =  contract_value.text
        self.published_date = published_date.text
        self.contract_date = contract_date.text
        self.completion_date = completion_date.text

The next function mines data from every single contract. It saves all the contract objects in a list called information.

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

driver = webdriver.Chrome('/Users/vatsalbajaj/Downloads/chromedriver')
information = []
def getInfo(link):
    try:
        str1 = link
        driver.get(str1)
        organisation_name = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[1]/td[@class = 'td_view_field']")
        tender_description = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[3]/td[@class = 'td_view_field']")
        tender_type = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[5]/td[@class = 'td_view_field']")
        number_of_bids = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[6]/td[@class = 'td_view_field']")
        name_of_selected_bidder = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[7]/td[@class = 'td_view_field']")
        contract_value = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[9]/td[@class = 'td_view_field']")
        published_date = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[10]/td[2]")
        contract_date = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[10]/td[4]")
        completion_date = driver.find_element_by_xpath("//table[@class = 'viewtablebg']/tbody/tr[11]/td[2]")
        information.append(Contract(organisation_name, tender_description, tender_type, number_of_bids, name_of_selected_bidder, contract_value, published_date, contract_date, completion_date))
    except:
        captcha_URL = driver.current_url
        driver.get(captcha_URL)
        captcha_problem = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/span").text
        answer = int(captcha_problem[0:2]) + int(captcha_problem[-4:-2])
        inputting_location = driver.find_element_by_xpath("//form/div/fieldset/div/div[2]/input")
        inputting_location.send_keys(answer)
        driver.find_element_by_xpath("//form/div/input").click()
        getInfo(link)
for link in links:
    getInfo(link)

## Load Libraries

In [170]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

Data cleaning process for every variable. First, we clean the tender type column with only the relevant tender types remaining.

In [171]:
tendertype_unique = []
for contract in information:
    tendertype_unique.append(contract.type)
print(list(set(tendertype_unique)))
tendertype = []
for tender in tendertype_unique:
    if tender == '' or tender == 'STE(OEM/OES)-SRM' or tender == '2' or tender == 'Single Tender(Urg-M)' or tender == '1' or tender == 'NA':
        tendertype.append(None)
    elif tender == 'Limted' or tender == 'LIMITED TENDER-SRM' or tender == 'LIMITED':
        tendertype.append('Limited')
    else:
        tendertype.append(tender)
print(list(set(tendertype)))
    

['', 'Limited', 'Services', 'STE(OEM/OES)-SRM', 'Limted', 'LIMITED TENDER-SRM', '1', 'Open', 'Works', '2', 'LIMITED', 'Single Tender(Urg-M)', 'NA', 'Goods']
['Limited', 'Open', 'Works', None, 'Services', 'Goods']


This function creates a dataframe and exports it.

In [172]:
from pandas import DataFrame
contracts = []
i = 0
for contract in information:
    if isinstance(contract.type, str) != True:
        contract.type = null
for contract in information:
    contracts.append([contract.organisation_name, contract.tender, tendertype[i]
                      , contract.number, contract.winner, contract.value, contract.published_date, 
                      contract.contract_date,  contract.completion_date])
    i += 1

    


In [173]:
df = pd.DataFrame(contracts, columns = ["Organisation name", "Tender description", "Tender type", "Number of bids", 
                                     "Name of selected bidder", "Contract value", "Published date", "Contract date"
                                     , "Completion date"])
df['Contract date'] = pd.to_datetime(df['Contract date'])  
df['Published date'] = pd.to_datetime(df['Published date'])  
df['Completion date'] = pd.to_datetime(df['Completion date'])
df['Number of bids'] = pd.to_numeric(df['Number of bids'], downcast = 'unsigned')
df['Contract value'] = pd.to_numeric(df['Contract value'])
df["Tender type"] = df["Tender type"].astype('category')

for completiondate in df['Completion date']:
    if isinstance(completiondate, dt.datetime) != True:
        completiondate = None


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 9 columns):
Organisation name          350 non-null object
Tender description         350 non-null object
Tender type                265 non-null category
Number of bids             346 non-null float64
Name of selected bidder    350 non-null object
Contract value             346 non-null float64
Published date             350 non-null datetime64[ns]
Contract date              350 non-null datetime64[ns]
Completion date            76 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](3), float64(2), object(3)
memory usage: 22.5+ KB


In [175]:
df

,Organisation name,Tender description,Tender type,Number of bids,Name of selected bidder,Contract value,Published date,Contract date,Completion date
0,NHPC Limited||Corporate Office-Faridabad-NHPC|...,Renewal of Annual Subscription of existing Sta...,Goods,1.0,Bentley Systems India Private Limited,505199.00,2017-12-29 15:00:00,2018-03-01 00:00:00,NaT
1,North Eastern Police Academy,"Supply, Installation, Testing, Commissioning o...",Works,2.0,Aspirify Enterprises Private Limited,1262600.00,2017-12-27 17:00:00,2018-03-19 00:00:00,NaT
2,Central Electricity Authority - CEA,Supply and Installation of Laptop Computers wi...,Goods,7.0,Corporate Infotech Pvt. Ltd,3332127.00,2017-12-27 12:00:00,2018-02-27 00:00:00,NaT
3,Department of Agricultural Research and Educat...,Software and Firewall Tender,Goods,5.0,"BIPS SYSTEMS LIMITED,Binary Semantics Limited,...",359901.68,2017-12-23 17:00:00,2018-03-07 00:00:00,NaT
4,Nuclear Power Corporation of India Limited,Comprehensive Annual Maintenance Contract for ...,NaN,1.0,Reprograhics India,477900.00,2017-12-22 17:00:00,2018-03-01 08:00:00,2019-02-28 08:00:00
5,Nuclear Power Corporation of India Limited,"Supply, Installation, Training and Warranty of...",NaN,3.0,Designtech Systems Limited,5900000.00,2017-12-22 14:30:00,2018-04-04 08:00:00,2018-05-02 08:00:00
6,Council of Scientific and Industrial Research|...,Supply and Installation of Upright Light Micro...,Goods,2.0,DSS IMAGETECH PVT. LTD.,559845.00,2017-12-21 18:30:00,2018-07-05 00:00:00,NaT
7,E-IN-C BRANCH - MILITARY ENGINEER SERVICES||CE...,QUOTATION FOR SUPPLY AND INSTALLATION OF ANTI ...,Goods,3.0,LAKSHYA INFOTECH,59850.00,2017-12-18 13:15:00,2018-03-06 00:00:00,NaT
8,NTPC Limited||Western Region - II Headquarter|...,Procurement of Softwares for NTPC-Khargone.,Goods,3.0,"ANB COMPUTER SOLUTIONS PRIVATE LIMITED,SAM SYS...",963208.00,2017-12-18 11:00:00,2018-03-12 00:00:00,NaT
9,BHEL Hyderabad,AMC FOR SIEMENS ENGG SOFTWARE PRODUCTS,NaN,0.0,SIEMENS INDUSTRY SOFTWARE INDIA PVT,0.00,2017-12-16 09:00:00,2017-12-18 09:00:00,2018-09-30 09:00:00


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 9 columns):
Organisation name          350 non-null object
Tender description         350 non-null object
Tender type                265 non-null category
Number of bids             346 non-null float64
Name of selected bidder    350 non-null object
Contract value             346 non-null float64
Published date             350 non-null datetime64[ns]
Contract date              350 non-null datetime64[ns]
Completion date            76 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](3), float64(2), object(3)
memory usage: 22.5+ KB


In [178]:
df

,Organisation name,Tender description,Tender type,Number of bids,Name of selected bidder,Contract value,Published date,Contract date,Completion date
0,NHPC Limited||Corporate Office-Faridabad-NHPC|...,Renewal of Annual Subscription of existing Sta...,Goods,1.0,Bentley Systems India Private Limited,505199.00,2017-12-29 15:00:00,2018-03-01 00:00:00,NaT
1,North Eastern Police Academy,"Supply, Installation, Testing, Commissioning o...",Works,2.0,Aspirify Enterprises Private Limited,1262600.00,2017-12-27 17:00:00,2018-03-19 00:00:00,NaT
2,Central Electricity Authority - CEA,Supply and Installation of Laptop Computers wi...,Goods,7.0,Corporate Infotech Pvt. Ltd,3332127.00,2017-12-27 12:00:00,2018-02-27 00:00:00,NaT
3,Department of Agricultural Research and Educat...,Software and Firewall Tender,Goods,5.0,"BIPS SYSTEMS LIMITED,Binary Semantics Limited,...",359901.68,2017-12-23 17:00:00,2018-03-07 00:00:00,NaT
4,Nuclear Power Corporation of India Limited,Comprehensive Annual Maintenance Contract for ...,NaN,1.0,Reprograhics India,477900.00,2017-12-22 17:00:00,2018-03-01 08:00:00,2019-02-28 08:00:00
5,Nuclear Power Corporation of India Limited,"Supply, Installation, Training and Warranty of...",NaN,3.0,Designtech Systems Limited,5900000.00,2017-12-22 14:30:00,2018-04-04 08:00:00,2018-05-02 08:00:00
6,Council of Scientific and Industrial Research|...,Supply and Installation of Upright Light Micro...,Goods,2.0,DSS IMAGETECH PVT. LTD.,559845.00,2017-12-21 18:30:00,2018-07-05 00:00:00,NaT
7,E-IN-C BRANCH - MILITARY ENGINEER SERVICES||CE...,QUOTATION FOR SUPPLY AND INSTALLATION OF ANTI ...,Goods,3.0,LAKSHYA INFOTECH,59850.00,2017-12-18 13:15:00,2018-03-06 00:00:00,NaT
8,NTPC Limited||Western Region - II Headquarter|...,Procurement of Softwares for NTPC-Khargone.,Goods,3.0,"ANB COMPUTER SOLUTIONS PRIVATE LIMITED,SAM SYS...",963208.00,2017-12-18 11:00:00,2018-03-12 00:00:00,NaT
9,BHEL Hyderabad,AMC FOR SIEMENS ENGG SOFTWARE PRODUCTS,NaN,0.0,SIEMENS INDUSTRY SOFTWARE INDIA PVT,0.00,2017-12-16 09:00:00,2017-12-18 09:00:00,2018-09-30 09:00:00


In [180]:
df.to_csv(r'/Users/vatsalbajaj/desktop/ccs/Procurement Research/procurement_data.csv')